## Assignments for Week-02

In this course, we learnt what's the search problem and what's the machine leanring. In this assignment, we need you do some more practice.

### 1. Re-code the house price machine learning

###### 1. Random Choose Method to get optimal *k* and *b*
###### 2.Supervised Direction to get optimal *k* and *b*
###### 3.Gradient Descent to get optimal *k* and *b*
###### 4. Try different Loss function and learning rate. 

For example, you can change the loss function: $Loss = \frac{1}{n} sum({y_i - \hat{y_i}})^2$ to $Loss = \frac{1}{n} sum(|{y_i - \hat{y_i}}|)$

And you can change the learning rate and observe the performance.

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1562414412237&di=8377c38d7b50c6f8d45b92ccc78dc1e7&imgtype=0&src=http%3A%2F%2Fmmbiz.qpic.cn%2Fmmbiz_jpg%2FlSyiavfm6wPgQDuDYa42IMgSkLOcPJU91RbkMibGy7VHLrhhNVLibTYk1iaLcmXKTdHc4icTDVia59QFXwqpW8cLflaw%2F640%3Fwx_fmt%3Djpeg)

## 2. Answer following questions:


###### 1. Why do we need machine learning methods instead of creating a complicated formula?


Ans:

###### 2.  Wha't's the disadvantages of `the 1st Random Choosen` methods in our course? 

Ans:

###### 3. Is the `2nd method supervised direction` better than 1st one?  What's the disadvantages of `the 2nd supversied directin` method? 

Ans:

###### 4. Why do we use `Derivative / Gredient` to fit a target function? 

Ans:因为梯度就是变量的变化方向，这样就避免的随机去找方向的，

###### 5. In the words 'Gredient Descent', what's the `Gredient` and what's the `Descent`?

Ans:Gradient 梯度， 多维空间里的导数；   Descent 下降，也就是按照梯度的方向每次都更新W值 

###### 6. What's the advantages of `the 3rd gradient descent method` compared to the previous methods?

Ans: 减少了随机性

###### 7. Using the simple words to describe: What's the machine leanring.

Ans:

## 3. Finish the search problem

Please using the search policy to implement an agent. This agent receives two input, one is @param start station and the other is @param destination. Your agent should give the optimal route based on Beijing Subway system. 

> Deadline: 2019-July-13

>Submit: Submit the source code and result to github. 



![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1562414356407&di=b57814aafd215bb8b8d9d8cd37c573d6&imgtype=0&src=http%3A%2F%2Fcli.clewm.net%2Ffile%2F2015%2F03%2F24%2F174ed60082b8422ac0636cfd3efb9e7f.jpg)

#### Dataflow: 

##### 1.	Get data from web page.

> a.	Get web page source from: https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485

> b.	You may need @package **requests**[https://2.python-requests.org/en/master/] page to get the response via url

> c.	You may need save the page source to file system.

> d.	The target of this step is to get station information of all the subway lines;

> e.	You may need install @package beautiful soup[https://www.crummy.com/software/BeautifulSoup/bs4/doc/]  to get the url information, or just use > Regular Expression to get the url.  Our recommendation is that using the Regular Expression and BeautiflSoup both. 

> f.	You may need BFS to get all the related page url from one url. 
Question: Why do we use BFS to traverse web page (or someone said, build a web spider)?  Can DFS do this job? which is better? 

使用BFS。 DFS也可以做，但是BFS会更好一点

In [17]:
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict

base_url = "https://baike.baidu.com"
headers = {"User-Agent" : "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;"}
html = requests.get(base_url + '/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485', headers=headers).text

soup = BeautifulSoup(html)

tds = soup.select("body > div.body-wrapper > div.content-wrapper > div > div.main-content > table:nth-child(66) tr td:nth-child(1)")

site_maps = defaultdict(list)

def get_sites(element, ):
    table_tds = element.select('tr td:nth-child(1)')
    sites = [td.text.strip() for td in table_tds if len(td.text.strip()) >= 2 and td.text.strip()[-1] == '站' ]

    if len(sites) < 1:
        table_tds = element.select('tr td:nth-child(2)')
        sites = [td.text.strip() for td in table_tds if len(td.text.strip()) >= 2 and td.text.strip()[-1] == '站' ]

    return sites

count= 1
for td in tds:
    a = td.find('a', href=True)
    if a:
        sub_url = td.a['href']
        if sub_url:
            count += 1
            print(str(count) + "开始爬取了的url"+sub_url)
            response = requests.get(base_url + sub_url, headers=headers)
            response.encoding ='utf-8'
            html_detail = response.text
            soup_detail = BeautifulSoup(html_detail)

            sites = []
            for caption in soup_detail.find_all('caption'):
                if "号线车站列表" in caption.text:
                    sites = get_sites(caption.parent)

                # 通过首末车时刻表来抓取
                if len(sites) < 1:
                    if "首末车时刻表" in caption.text:
                        table_tds = caption.parent.select('tr th:nth-child(1)')
                        sites = [td.text.strip() for td in table_tds]
                        sites = [site for site in sites if site != '车站名称']

                if len(sites) > 0:
                    break


            # https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%8116%E5%8F%B7%E7%BA%BF
            if len(sites) < 1:
                tables = soup_detail.find_all('th', text="车站名称")
                if len(tables) > 0:
                    sites = get_sites(tables[0].parent.parent)

                    if len(sites) == 0:
                        sites = tables[-1].parent.parent.select('tr th:nth-child(1)')
                        sites = [td.text.strip() for td in table_tds]
                        sites = [site for site in sites if site != '车站名称']

            # https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81%E4%BA%A6%E5%BA%84%E7%BA%BF
            # 还有问题

            if len(sites) > 0:
                print(str(count) + "爬取到数据的url" + sub_url)

            sites = [site if '站' == site[-1] else site + '站' for site in sites if len(site) <= 10 and len(site) >= 2]
            for i in range(len(sites) - 1):
                site_maps[sites[i]].append(sites[i + 1])
                site_maps[sites[i + 1]].append(sites[i])


print(site_maps)

2开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%811%E5%8F%B7%E7%BA%BF
2爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%811%E5%8F%B7%E7%BA%BF
3开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%812%E5%8F%B7%E7%BA%BF
3爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%812%E5%8F%B7%E7%BA%BF
4开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%814%E5%8F%B7%E7%BA%BF
4爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%814%E5%8F%B7%E7%BA%BF
5开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%815%E5%8F%B7%E7%BA%BF
5爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%815%E5%8F%B7%E7%BA%BF
6开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%816%E5%8F%B7%E7%BA%BF
6爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%816%E5%8F%B7%E7%BA%BF
7开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%817%E5%8F%B7%E7%BA%BF
7爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%817%E5%8F%B7%E7%BA%BF
8开始爬取了的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%818%E5%8F%B7%E7%BA%BF/897830
8爬取到数据的url/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%818%E5%

##### 2.	Preprocessing data from page source.

> a.	Based on the page source gotten from url. You may need some more preprocessing of the page. 

> b.	the Regular Expression you may need to process the text information.

> c.	You may need @package networkx, @package matplotlib to visualize data. 

> d.	You should build a dictionary or graph which could represent the connection information of Beijing subway routes. 

> e.	You may need the defaultdict, set data structures to implement this procedure. 

In [22]:
def search(start, end):
    if start not in site_maps:
        return '起始点错误'
    
    visited = set()
    routes = [[start]]
    res = []
    while routes:
        route = routes.pop(0)
#         print(route)
#         print(visited)
        current = route[-1]
        if current in visited: continue    
            
        for site in site_maps[current]:
            if site in route: continue
                
            new_route = route + [site]
            if site == end:
                res.append(new_route)
            else:   
                routes.append(new_route)
            
        visited.add(current)

    return res

def wrapper_print(routes):
    for route in routes:
        print('->'.join(route))
    
def get_shortest_route(routes):
    return sorted(routes, key=len)

wrapper_print(get_shortest_route(search('积水潭站', '安定门站')))
wrapper_print(search('知春里站', '东直门站'))
            

积水潭站->鼓楼大街站->安定门站
积水潭站->鼓楼大街站->什刹海站->南锣鼓巷站->东四站->张自忠路站->北新桥站->雍和宫站->安定门站
知春里站->知春路站->大钟寺站->明光桥西站->西直门站->积水潭站->鼓楼大街站->安定门站->雍和宫站->东直门站
知春里站->知春路站->西土城站->牡丹园站->健德门站->北土城站->安贞门站->惠新西街南口站->芍药居站->光熙门站->柳芳站->东直门站
知春里站->知春路站->大钟寺站->明光桥西站->西直门站->积水潭站->鼓楼大街站->什刹海站->南锣鼓巷站->东四站->朝阳门站->东四十条站->东直门站


##### 3. Build the search agent

> Build the search agent based on the graph we build.

for example, when you run: 

```python
>>> search('奥体中心', '天安门') 
```
you need get the result: 

奥体中心-> A -> B -> C -> ... -> 天安门



## （Optional）Create different policies for transfer system.


以下部门为可选部分，请酌情完成。 并不要求全部同学完成。

As much as you can to use the already implemented search agent. You just need to define the **is_goal()**, **get_successor()**, **strategy()** three functions. 

> a.	Define different policies for transfer system. 

> b.	Such as Shortest Path Priority（路程最短优先）, Minimum Transfer Priority(最少换乘优先), Comprehensive Priority(综合优先)

> c.	Implement Continuous transfer. Based on the Agent you implemented, please add this feature: Besides the @param start and @param destination two stations, add some more stations, we called @param by_way, it means, our path should from the start and end, but also include the  @param by_way stations. 

e.g 
```
1. Input:  start=A,  destination=B, by_way=[C] 
    Output: [A, … .., C, …. B]
2. Input: start=A, destination=B, by_way=[C, D, E]
    Output: [A … C … E … D … B]  
    # based on your policy, the E station could be reached firstly. 
![image.png](attachment:image.png)
```

##### 5.	Test your result with commercial applications. 

将你的结果和高德地图或者百度地图进行比较，如果有不同，请分析原因


恭喜，完成本次课程，你对常用的人工智能方法以及有一定的了解了。基于规则的，基于概率模型的，基于搜索的，基于机器学习的。 可以说，我们现在通常见到的方法都能够归属到这几类方法中。 这就是**人工智能**，并没有很难是吧？ 继续加油！

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1562415163815&di=4b29a2a863a8285212033760f288ed7a&imgtype=0&src=http%3A%2F%2F5b0988e595225.cdn.sohucs.com%2Fimages%2F20180710%2F8704194a1d7f46a383ddc29d40c9bca5.jpeg)